# Part 0: preliminaries

1. course evaluations are out

2. suggest review session topics on Ed

In [ ]:
%pip install plotly

# Part 1: interactive 3D plotting

In [ ]:
# import libraries

import plotly.express as px

import pandas as pd
import numpy as np

In [ ]:
# generate synthetic data

n = 100
r = np.random.uniform(0, 5, size=n)
t = np.random.uniform(0, 2*np.pi, size=n)
y = (r < 2.5).astype(str)

X = pd.DataFrame({
    "x1": r * np.cos(t),
    "x2": r * np.sin(t),
    "y": y
})

X

In [ ]:
fig = px.scatter(
    X,
    x = "x1", y = "x2", color = "y"
)
fig.update_layout(yaxis_scaleanchor="x") # force equal aspect ratio

In [ ]:
X["x3"] = ... # your code here 

fig_3d = px.scatter_3d(
    X,
    x = "x1", y = "x2", z = "x3",
    color = "y",
)

fig_3d.update_traces(marker=dict(size=5))
fig_3d.update_layout(yaxis_scaleanchor="x")


# Part 2: higher-level fine-tuning tools

## increasing abstraction

### PS4: implement gradient descent from scratch
```python
for _ in range(max_iter):
    theta = theta - learning_rate * gradient(loss, theta)
```

### PS6: implement training steps from scratch
```python
model = MyCoolNeuralNetwork()
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.SGD()

for epoch in range(num_epochs):
    model.train()
    for (X, y) in training_data_batch:
        y_hat = model(X)

        loss = loss_function(y, y_hat)
        loss.backwards()

        optimizer.step()
    
    model.eval()
    for (X, y) in validation_data_batch:
        ...
```

### PS7: higher-level APIs akin to sklearn's `model.fit()`/`model.predict()`

## options
- for Transformer models, [`transformers`](https://huggingface.co/docs/transformers/en/main_classes/trainer) has a number of automated training tools (see [LLM fine-tuning guide](https://huggingface.co/learn/llm-course/en/chapter3/3))

- for generic pytorch models, you can use tools like [pytorch lightning](https://lightning.ai/docs/pytorch/stable/)

- for computer vision model definitions, you've already seen `torchvision`. other options include: [`timm`](https://timm.fast.ai/), which has a similar API to the `transformers` library we'll use below and in PS7.

In [ ]:
%pip install timm transformers datasets evaluate "transformers[torch]"

In [ ]:
# from torchvision import datasets, transforms
import torchvision as tv
import torch.nn as nn
import numpy as np

def load_cifar10(data_path = "."):
    """
    Helper code to clean the CIFAR 10 dataset, and remove the unnecessary classes.
    """
    class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

    cifar10 = tv.datasets.CIFAR10(
        data_path, train=True, download=True,
        transform=tv.transforms.ToTensor
    )

    cifar10_val = tv.datasets.CIFAR10(
        data_path, train=False, download=True,
        transform=tv.transforms.ToTensor
    )

    return cifar10, cifar10_val

cifar10, cifar10_val = load_cifar10()



label_map = {1: 0, 9: 1}
class_names = ['car', 'truck']
cifar2 = [(img, label_map[label]) for img, label in cifar10 if label in list(label_map.keys())]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in list(label_map.keys())]



In [ ]:
import evaluate
import numpy as np
import torch
from datasets import load_dataset
from PIL import Image
from transformers import TrainingArguments, Trainer, ViTForImageClassification, ViTImageProcessor


# Load the CIFAR-10 dataset and filter for car and truck classes
label_map = {1: 0, 9: 1}
class_names = ['car', 'truck']

cifar_subset = load_dataset("cifar10", split="train")
cifar_subset_val = load_dataset("cifar10", split="test")

# Load the pre-trained ViT model and image processor
VIT_MODEL = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(VIT_MODEL)
model = ViTForImageClassification.from_pretrained(
    VIT_MODEL,
    num_labels=len(class_names), ignore_mismatched_sizes=True,
    id2label={i: c for i, c in enumerate(class_names)},
    label2id={c: i for i, c in enumerate(class_names)}
)

# Define the evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=predictions, references=labels)

# Preprocess function to convert images to format model expects
def preprocess_function(examples):
    inputs = image_processor(examples["img"], return_tensors="pt")
    inputs["labels"] = [label_map[label] for label in examples["label"]]

    return inputs

# Apply the preprocessing function to the datasets
cifar_processed = cifar_subset\
    .filter(lambda _: _["label"] in list(label_map.keys()))\
    .map(preprocess_function, batched=True, remove_columns=cifar_subset.column_names)

cifar_processed_val = cifar_subset_val\
    .filter(lambda _: _["label"] in list(label_map.keys()))\
    .map(preprocess_function, batched=True, remove_columns=cifar_subset.column_names)


In [ ]:
# Define the training arguments
hyperparameters = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=...,
    per_device_eval_batch_size=...,
    num_train_epochs=...,
    learning_rate=...,
    weight_decay=...,
    remove_unused_columns=False,
    report_to="none"
)

# Create the Trainer and train the model
trainer = Trainer(
    model=model,
    args=hyperparameters,
    train_dataset=cifar_processed,
    eval_dataset=cifar_processed_val,
    compute_metrics=compute_metrics,
)

trainer.train()